In [1]:
'''
MathU is a math training program for first-grade kids.

It presents both arithmetic problems and number comparison problems.
It uses a graphical interface and sound effects to make using it more engaging.
The comparison problems have yes/no answers.
The arithmetic problems are presented in a multiple choice format.
'''

import tkinter as tk                                                                    # import gui manager
from tkinter import messagebox                                                          # import messagebox separately
import time, logging, os, sys, PIL                                                      # import system kinds of things
import random                                                                           # import random en masse
from PIL import Image, ImageTk

# Class and method definitions

# Google Colab is not supported due to both tkinter and playsound3 issues
colab = 'google.colab' in sys.modules                                                   # see if we are/are not running under colab
if colab: raise(Exception, 'Google Colab is not supported because of TK terminal access requirements')

# Class Problem handles generic question/answer, status/progress and operands, operators, results
class MathProblem:           
    # constructor
    def __init__(self): raise Exception('Class MathProblem is not supported for object creation')

    # initializer for dependent class objects
    def _init_(self, type, operatorList):        
        # define general problem attributes
        self.question = None                                                             # problem question is set by inheritor
        self.answer = None                                                               # problem answer is set by inheritor
        self.attempted = False                                                           # a nonblank answer attempt has been made
        self.solved = False                                                              # the correct answer was given
        self.guess = ''                                                                  # the most recent attempt at the answer
        self.guesses = 0                                                                 # the number of attempts tried so far
        self.answerType = ''
        self.type = type                                                                # indicate problem type
        self.getOperator(operatorList)                                                  # get a random (binary) operator from list
        
        self.getOperandPair()                                                           # get two whole number operands

    # Get an operator based on the list of allowed operators passed in
    def getOperator(self, operatorList): 
        self.operator = random.choice(operatorList)

    # Get operands - two random integers since all operators are binary
    def getOperandPair(self, operandMax = 40): 
        self.operands = [random.randint(0, operandMax), random.randint(0, operandMax)]

# class to represent an arithmetic math problem: addition, subtraction, or multiplication
class ArithmeticProblem(MathProblem):
    # constructor
    def __init__(self):
        # Set overall attributes and pick a random arithmetic operator
        super()._init_('arithmetic', ('+', '-', 'x'))                                   # call parent (MathProblem) initializer

        # if needed, simplify the problem some by tweaking operands
        if self.operator == '-' and self.operands[0] < self.operands[1]:                # if subtraction and difference is negative
            self.operands[0], self.operands[1] = self.operands[1], self.operands[0]     #     swap the two operands
            
        elif self.operator == 'x' and self.operands[0] * self.operands[1] > 100:        # if multiplication and the product is > 100
            self.getOperandPair(20)                                                     #     get two smaller operands
            if self.operands[0] * self.operands[1] > 100: 
                self.getOperandPair(10)                                                 #     if still > 100 get two even smaller operands

        # Put the problem into a question form
        self.mathProblem = f'{self.operands[0]:2} {self.operator:1} {self.operands[1]:2}' 
        self.question = f'What is {self.operands[0]} {self.operator} {self.operands[1]} ?' 

        # Get the answer to the question above
        match self.operator:
            case '+': self.answer = self.operands[0] + self.operands[1]
            case '-': self.answer = self.operands[0] - self.operands[1]
            case 'x': self.answer = self.operands[0] * self.operands[1]
        self.answerType = 'a whole number'

    # Method to check if a guess is correct and return True or False
    def isCorrect(self):
        if not self.guess: return False
        try: self.solved = True if int(self.guess) == self.answer else False
        except ValueError: return False
        return self.solved
    
    # Method to return a hint at why an incorrect answer was wrong
    def hint(self):
        if self.operator   == '+' and 0  in self.operands: return "A number plus 0 doesn't change"
        elif self.operator == '-' and 0  in self.operands: return "A number minus 0 doesn't change"
        elif self.operator == 'x' and 0  in self.operands: return 'A number times 0 is 0'
        elif self.operator == 'x' and 1  in self.operands: return "A number times 1 doesn't change"
        elif self.operator == 'x' and 10 in self.operands: return 'A number times 10 is that number with a 0'
        elif self.operator == 'x' and 5  in self.operands: return 'A number times 5 ends in either 0 or 5'
        elif self.operator == '-' and self.operands[0] == self.operands[1]: return 'A number minus itself is 0'
        elif not self.guess.isdigit(): return f'{self.guess} is not a whole number'
        return f'{self.guess} is too little' if int(self.guess) < self.answer else f'{self.guess} is too big'

# class to represent a comparison math problem: (not)equal, less than (or equal), greater than (or equal)
class CompareProblem(MathProblem):    
    # constructor
    def __init__(self):
        # Set overall attributes and pick a random numerical comparison operator
        super()._init_('comparison', ('=', '≠', '<', '>', '≤', '≥'))      # call parent (MathProblem) initializer

        # Put the problem into a question form
        self.mathProblem = f'{self.operands[0]:2} {self.operator:1} {self.operands[1]:2}' 
        self.question = f'Is {self.operands[0]} {self.operator} {self.operands[1]} ?' 

        # Evaluate the answer to the comparision question
        match self.operator:
            case '=': self.answer = 'yes' if self.operands[0] == self.operands[1] else 'no'
            case '≠': self.answer = 'yes' if self.operands[0] != self.operands[1] else 'no'
            case '<': self.answer = 'yes' if self.operands[0] <  self.operands[1] else 'no'
            case '>': self.answer = 'yes' if self.operands[0] >  self.operands[1] else 'no'
            case '≤': self.answer = 'yes' if self.operands[0] <= self.operands[1] else 'no'
            case '≥': self.answer = 'yes' if self.operands[0] >= self.operands[1] else 'no'
        self.answerType = 'yes or no'
   
    # Method to check if a guess is correct and return True or False
    def isCorrect(self):
        if not self.guess: return False
        self.solved = True if self.answer.startswith(self.guess.lower()) else False
        return self.solved
    
    # Method to return a hint at why an incorrect answer was wrong
    def hint(self):
        match self.operator:
            case '=': return f'Are the two numbers the same?'
            case '≠': return f'Are the two numbers different?'
            case '<': return f'Is {self.operands[0]} less than {self.operands[1]}?'
            case '>': return f'Is {self.operands[0]} more than {self.operands[1]}?'
            case '≤': return f'Is {self.operands[0]} the same or less than {self.operands[1]}?'
            case '≥': return f'Is {self.operands[0]} the same or more than {self.operands[1]}?'
            
# class to play sounds stored as mp3 files saved in the MathU.data subdirectory
class Mp3:
    from playsound3 import playsound
    
    # determine what directory contains our mp3 files
    def __init__(self, mp3Dir = None):
        # determine directory storing the mp3 files
        self.dir = os.getcwd()
        mp3SubDir = 'MathU.data'
        if mp3Dir is not None: 
            self.mp3Dir = mp3Dir
        elif '__file__' in globals() and os.path.exists(f'{os.path.dirname(__file__)}\\{mp3SubDir}'):
            self.dir = os.path.dirname(__file__)
            self.mp3Dir = f'{self.dir}\\{mp3SubDir}'
        elif os.path.exists(f'{os.path.dirname(sys.argv[0])}\\{mp3SubDir}'):
            self.dir = os.path.dirname(sys.argv[0])
            self.mp3Dir = f'{self.dir}\\{mp3SubDir}'
        elif os.path.exists(f'{os.getcwd()}\\{mp3SubDir}'):
            self.dir = os.getcwd()
            self.mp3Dir = f'{self.dir}\\{mp3SubDir}'
        else: 
            self.mp3Dir = None

    # method to return the directory we are playing mp3 files from
    def __str__(self):
        return f'Playing mp3 files from directory "{self.mp3Dir}"'
        
    # method for playing an mp3 whose filename is passed
    def play(self, mp3File, block=False):
        if self.mp3Dir is not None: 
            try: 
                Mp3.playsound(f'{self.mp3Dir}\\{mp3File}.mp3',block) 
            except Exception as e:                                                          # if something went wrong
                self.mp3Dir = None                                                          #    disable playing more sounds
                logging.info(f' {e}. Sounds disabled.')

# class to drive the main problem session
class MathU:
    def __init__(self):
        logging.basicConfig(level=logging.INFO)                                             # set up logging

        # determine our program directory 
        if '__file__' in globals() and os.path.exists(f'{os.path.dirname(__file__)}'): 
            self.dir = os.path.dirname(__file__)
        elif os.path.exists(f'{os.path.dirname(sys.argv[0])}'): 
            self.dir = os.path.dirname(sys.argv[0])
        else: self.dir = os.getcwd()
            
        # get sound player and play a "hello" sound 
        self.mp3 = Mp3()                                                                    # Get an Mp3 object to play sounds
        self.mp3.play('hellothere')                                                         # Say hello
        
        self.problems = []                                                                  # list of problems worked on
        
        self.gui = tk.Tk()                                                                  # get the main gui window
        self.gui.protocol('WM_DELETE_WINDOW', self.quitMathU)                               # handle end-window ('X') clicked
        self.gui.bind('<Escape>', self.quitMathU)                                           # handle Esc key pressed
    
        # Build student name prompt with splash image
        # get a frame to hold the name question
        self.gui.geometry('300x690+500+100')
        self.gui.title('MathU Hello')
        
        self.nameFrame = tk.Frame(self.gui)
        self.nameFrame.grid()

        # get a label for the input field
        self.nameLabel = tk.Label(self.nameFrame, font=('Arial', 18, 'bold'), fg='green',
                                  text=f"\nMy name is Matthew.\nWhat's your name?")
        self.nameLabel.grid(row=0, column=0, pady=5)

        # get the input field
        self.nameEntry = tk.Entry(self.nameFrame, width=20, font=('Arial', 18, 'bold'))
        self.nameEntry.grid(padx=10, row=1, column=0)
        self.nameEntry.bind('<Return>', self.nameEnterKey)                                  # handle Enter key pressed
        self.gui.attributes('-topmost', True)                                               # ...
        self.nameEntry.focus_force()                                                        # ...

        # get the start and quit buttons
        self.nameButtons = tk.Frame(self.nameFrame)
        self.nameButtons.grid()
        
        self.nameStart = tk.Button(self.nameButtons, text='Start', width=5, font= ('Arial', 12, 'bold'), bg='green', 
                                   command=self.nameEnterKey)
        self.nameStart.grid(pady=10, row=2, column=0, padx=10)
        
        self.nameQuit = tk.Button(self.nameButtons, text='Quit', width=5, font= ('Arial', 12, 'bold'), bg='red', 
                                  command=self.quitMathU)
        self.nameQuit.grid(pady=10, row=2, column=1, padx=10)
    
        # show splash screen image
        try:
            image = Image.open(f'{self.dir}\\MathU.data\\splash.png')
#            image = image.resize((250, 500), Image.LANCZOS)
            image = ImageTk.PhotoImage(image)
            self.splash = tk.Label(self.nameFrame, image=image)
            self.splash.image = image
            self.splash.grid()
        except: self.gui.geometry('300x200+500+100')

    # Event handler for the Enter key in name entry field
    def nameEnterKey(self, event=None):
        # capitalize the first letter of each word of the name and save the name
        self.name = ' '.join([word[:1].upper() + word[1:] for word in self.nameEntry.get().split()])
        if not self.name: self.name = 'Math Student'                                        # use 'Math Student' if no name entered
        self.nameFrame.destroy()                                                            # get rid of name prompt
        self.question1()                                                                    # set up first math question

    # Set up the main session and the first math question to prime the gui event loop
    def question1(self):
        # reset gui window and title
        self.gui.geometry('525x435+400+200')
        self.gui.title(f'MathU {self.name}')
        self.mp3.play('letsgo')
        
        # get the first math problem and put it on the problem list
        self.problem = ArithmeticProblem() if random.choice((0, 1)) else CompareProblem()
        self.problems.append(self.problem)                                                

        # create a frame to contain math problem widgets
        self.mathFrame = tk.Frame(self.gui)
        self.mathFrame.grid()
        
        # create two labels - one for the question and one for a prompt to click the right answer button
        self.mathQuestion = tk.Label(self.mathFrame,text=f'\n{self.problem.mathProblem}',width=20,font=('Arial',20,'bold'),fg='blue')
        self.mathQuestion.grid()
        self.mathQuestion2 = tk.Label(self.mathFrame, height=2, text=f'Click on the answer', font=('Arial',16,'bold'),fg='blue')
        self.mathQuestion2.grid()
        
        # create the yes / no buttons for comparison selections
        self.buttonsYesNo = tk.Frame(self.mathFrame)
        self.buttonsYesNo.grid()
        
        self.buttonYes = tk.Button(self.buttonsYesNo, text='yes', width=4, font=('Arial',14,'bold'),
                                 command=lambda: self.checkSelection("yes"))
        self.buttonYes.grid(row=0, column=0, padx=25)
        
        self.buttonNo = tk.Button(self.buttonsYesNo,text='no',width=4,font=('Arial',14,'bold'),
                                 command=lambda: self.checkSelection("no"))
        self.buttonNo.grid(row=0, column=1, padx=25)
        
        self.buttonsYesNo.grid_remove()                                                          # hide yes/no buttons for now
        
        # create multiple-choice buttons for arithmetic selections
        self.buttonsChoice = tk.Frame(self.mathFrame)
        self.buttonsChoice.grid()
        
        self.buttonChoiceList = [] 
        for i in range(6):
            buttonChoice = tk.Button(self.buttonsChoice, width=4, font=('Arial',14,'bold'),
                                command=lambda i=i: self.checkSelection(self.buttonChoiceList[i].cget("text")))
            buttonChoice.grid(row=0, column=i, padx=5)
            self.buttonChoiceList.append(buttonChoice)

        # create a label for feedback info
        self.mathFeedBack = tk.Label(self.mathFrame, height=2, width=35, font=('Arial', 18, 'bold'))
        self.mathFeedBack.grid()

        # create buttons to ask to show the answer / get a new problem, or quit
        self.endButtons = tk.Label(self.mathFrame)
        self.endButtons.grid(row=6, pady=10)
        
        self.buttonShowNew = tk.Button(self.endButtons, text='Show', bg='yellow',width=5,
                                       font=('Arial', 12, 'bold'), command = lambda : self.checkSelection('Show'))
        self.buttonShowNew.grid(row=0, column=0, padx=5)
        
        # show mini-splash screen image
        try:
            image = Image.open(f'{self.dir}\\MathU.data\\mini.png')
#            image = image.resize((100, 190), Image.LANCZOS)
            image = ImageTk.PhotoImage(image)
            self.mini = tk.Label(self.endButtons, image=image)
            self.mini.image = image
            self.mini.grid(row=0, column=1, padx=30)
        except: self.gui.geometry('525x280+400+200')

        self.buttonQuit = tk.Button(self.endButtons, text='Quit', bg='red', width=5,
                                    font=('Arial',12,'bold'), command=self.quitMathU)
        self.buttonQuit.grid(row=0, column=2, padx=5)
        
        # display the answer selection buttons for the first math problem
        self.displayProblemButtons()

    # display answer selections buttons appropriate to the current problem
    def displayProblemButtons(self):
        if self.problem.type == 'arithmetic':
            # hide comparison buttons and show arithmetic buttons
            self.buttonsYesNo.grid_remove()                                                   # hide comparison buttons
            self.buttonsChoice.grid()                                                         # show arithmetic buttons
            
            # rebuild the button text to use the correct problem answer plus random numbers
            self.selectionList = [self.problem.answer]                                        # initialize list with correct answer
            while len(self.selectionList) < len(self.buttonChoiceList):                       # add some random answers into the list
                selection = random.randint(0, 80)
                # add random selection chosen to the selection list unless it is a dup
                if selection not in self.selectionList:
                    self.selectionList.append(selection)       
            random.shuffle(self.selectionList)
            for i in range(len(self.buttonChoiceList)):                                       # copy the answer choices to button text
                self.buttonChoiceList[i].configure(text=str(self.selectionList[i]))
            
        else: # comparison problem
            # hide arithmetic buttons and show comparison buttons
            self.buttonsChoice.grid_remove()                                                  # hide arithmetic buttons
            self.buttonsYesNo.grid()                                                          # show comparison buttons
        
    # check user selection
    # (this is the main driver for asking questions and handling answer selections)
    def checkSelection(self, selection):
        self.buttonShowNew.configure(text='Show', bg='yellow')
        
        # if they solved the problem or ran out of chances, get a new problem
        if self.problem.solved or self.problem.guesses == 3:
            self.problem = ArithmeticProblem() if random.choice((0, 1)) else CompareProblem()  # Get a new problem to solve
            self.problems.append(self.problem)                                                 # Add problem to problem list
            
            # reset gui fields for the new problem
            self.mathQuestion.configure(text=f'\n{self.problem.mathProblem}')                    # Set new question
            self.mathQuestion2.configure(text=f'Click on the answer')                             # Prompt to click on answer
            
            self.mathFeedBack.configure(text='', height=2)                                     # Clear feedback area
            
            # show the selection buttons for the new problem
            self.displayProblemButtons()
        
        else:
            # they are already working on a problem
            self.problem.attempted = True                                                       # they tried at least once
            self.problem.guesses += 1                                                           # increment number of tries
            self.problem.guess = selection                                                      # remember most recent try

            # did they get the correct answer?
            if self.problem.isCorrect():
                # remove (hide) the selection buttons and the prompt for clicking the selection buttons
                self.buttonsYesNo.grid_remove()
                self.buttonsChoice.grid_remove()
                self.mathQuestion2.configure(text='')
                
                # tell them they got the answer right
                emoji = random.choice(('\U0001f600', '\U0001f601', '\U0001f973', '\U0001f389')) # get a random happy emoji
                self.mathFeedBack.configure(text=f'Correct, the answer is {self.problem.answer}! {emoji}\n', fg='green', height=3)
                self.buttonShowNew.configure(text='New', bg='green')                            # change 'Show' (answer) to 'new' (problem)
                self.mp3.play(f'{random.choice(('nice', 'yes', 'heisgood'))}')                  # play a random happy sound
            
            else:  # a wrong answer button was selected
                # did they use all 3 tries, or ask for the answer early by clicking the 'Show' button?
                if self.problem.guesses == 3 or selection == 'Show':
                    self.problem.guesses = 3                                                    # if they selected "show", pretend it was last try
                    
                    self.buttonsYesNo.grid_remove()                                             # remove yes/no buttons
                    self.buttonsChoice.grid_remove()                                            # remove multiple-choice buttons
                    
                    self.mathQuestion2.configure(text='')                                          # remove prompt to click on selection
                    self.mathFeedBack.configure(height=3)
                    
                    self.buttonShowNew.configure(text='New', bg='green')                        # prepare prompt for a new problem
                    self.mathQuestion2.configure(text='')
                    self.mathFeedBack.configure(text=f'The answer is {self.problem.answer}.\n', fg='red')  # reveal answer
                    
                    self.mp3.play('ohwell')
                
                # they got it wrong this time but they still have more chances, so set up the next one
                else:
                    self.mathFeedBack.configure(text = self.problem.hint(), fg='red')           # give them a hint on solving
                    self.mp3.play('uhuh')
                    
    # Handler for click on 'Quit' button or window 'X' to end MathU processing
    def quitMathU(self, event=None):                                                            # event=None if click on 'x'
        countA, countC, countM = self.getSummary()                                              # accumulate problem counts
        #  self.parentReport(countA, countC, countM)                                            # skip call to report for now
        
        self.gui.withdraw()                                                                     # make window disappear
        self.byeMessage(countM)                                                                 # goodbye popup messagebox and sounds
        
        try: self.gui.destroy()                                                                 # get rid of gui interface
        except: pass                                                                            # ignore any errors on the way out

    # Accumulate summary counts about this problem solving session
    def getSummary(self):
        # [0]=asked, [1]=skipped, [2]=attempted, [3]=wrong, [4]=correct, [5-7]=correct on 1st-3rd try                
        countA = [0, 0, 0, 0, 0, 0, 0, 0]                                                       # init arithmetic counts
        countC = [0, 0, 0, 0, 0, 0, 0, 0]                                                       # init comparison counts
        countM = [0, 0, 0, 0, 0, 0, 0, 0]                                                       # init math total counts
        for problem in self.problems:
            # Arithmetic problem counts
            if problem.type == 'arithmetic':                                                  
                countA[0] += 1                                                                  # asked
                countA[1] += 1 if not problem.attempted else 0                                  # skipped
                countA[2] += 1 if problem.attempted else 0                                      # attempted
                if problem.attempted and not problem.solved:
                    countA[3] += 1                                                              # tried but not solved
                if problem.solved:                                                              # if they got it correct
                    countA[4] += 1                                                                # one of the attempts was correct
                    countA[problem.guesses + 4] += 1                                              # and it was this one                    
            # Comparison problem counts
            else:                                                                               
                countC[0] += 1                                                                  # asked
                countC[1] += 1 if not problem.attempted else 0                                  # skipped
                countC[2] += 1 if problem.attempted else 0                                      # attempted
                if problem.attempted and not problem.solved:
                    countC[3] += 1                                                              # tried but not solved
                if problem.solved:                                                              # if they got it correct
                    countC[4] += 1                                                                # one of the attempts was correct
                    countC[problem.guesses + 4] += 1                                              # and it was this one                    
        # get math total counts (arithmetic counts plus comparison counts)
        for i in range(len(countM)):
            countM[i] = countA[i] + countC[i]
        return countA, countC, countM
        
    # Display goodbye popup message
    def byeMessage(self, countM):
        endMessage = None
        if countM[2] > 1 and countM[4] > countM[2] / 2:                                         # More than half of tried problems right?
            if countM[0] == countM[5]:                                                          # All asked problems right first time?
                endMessage = f'You got all {countM[0]} right the first time! Most excellent, {self.name}!'
                self.mp3.play('heisgood')
            elif countM[0] == countM[4]:                                                        # All asked problems right?
                endMessage = f"You didn't miss any of {countM[0]} asked! Excellent, {self.name}!"
                self.mp3.play('heisgood')
            elif countM[2] == countM[4]:                                                        # All tried problems right?
                endMessage = f"You didn't miss any of {countM[2]} tried! Great work, {self.name}!"
                self.mp3.play('heisgood')
            else:                                                                               # Got more than half tried right, but not all
                endMessage = f'You got {countM[4]} right out of {countM[2]} tried! Nice job, {self.name}!'
                self.mp3.play('heisgood')            
        elif countM[2] > 0:                                                                     # At least one correct?
            endMessage = f'{self.name}, you got {countM[4]} right out of {countM[2]} tried.'
            self.mp3.play('goodbye')
        else:
            self.mp3.play('goodbye', True)
        if endMessage:
            messagebox.showinfo('info', endMessage, icon='question')                            # icon='question' stops alert tone

    # Write report for parent / teacher (not called for now)
    def parentReport(self, countA, countC, countM):
        if countM[0] <= 1 and countM[2] == 0:
            return

        fileName = f'{self.dir}\\MathU.ParentReport.{self.name[0:15]}.{time.strftime('%Y%m%d%H%M%S', time.localtime())}.txt'
        try:
            with open(fileName, 'w', encoding = 'UTF-8') as parentReport:
                parentReport.write(f'''\t\t\tMathU Problem Summary
           Problems  Skipped   Answered  Wrong    Correct  First   Second  Third
Arithmetic {countA[0]:4}      {countA[1]:4}      {countA[2]:4}      {countA[3]:4}     {countA[4]:4}     {countA[5]:4}    {countA[6]:4}    {countA[7]:4}
Comparison {countC[0]:4}      {countC[1]:4}      {countC[2]:4}      {countC[3]:4}     {countC[4]:4}     {countC[5]:4}    {countC[6]:4}    {countC[7]:4}
Math Total {countM[0]:4}      {countM[1]:4}      {countM[2]:4}      {countM[3]:4}     {countM[4]:4}     {countM[5]:4}    {countM[6]:4}    {countM[7]:4}\n''')
                if len(self.problems) > 0:
                    parentReport.write(f'\n\nProblems asked:')
                    for p in self.problems:
                        parentReport.write(f'\n{p.type}: {p.mathProblem:7}, ans={p.answer:3}, reply={p.guess:3}, ok={p.solved}')
        except Exception as e:
            logging.error(f'Error writing report to {self.dir}: {e}')
        
if __name__ == '__main__':                                                                      # if non-import (normal) case
    MathU().gui.mainloop()                                                                      #    start main gui driver loop
